In [1]:
## classique 

import numpy as np
import torch 
import h5py # pour gérer les formats de données utilisés ici 
import torch
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import utils

## handmade


On commence par charger les données d'entraînement, que l'on va plus tard utiliser pour entraîner le modèle de few-shot learning en question. 

In [2]:
# Chemin vers le fichier de données
train_data_path = '/Users/badis/MVA_deep_and_signal/Few_shots_classif/train.hdf5'

# Chargement des données HDF5
with h5py.File(train_data_path, 'r') as data:
    x_train = torch.tensor(data['signaux'][:])  # Convertir les signaux en tenseur PyTorch
    # Forme [30000, 2048, 2] que l'on réorganise en [30000, 2, 2048] : (batch, canal, résolution_temporelle)
    x_train = x_train.permute(0, 2, 1)  
    snr_train = torch.tensor(data['snr'][:])    
    y_train = torch.tensor(data['labels'][:])  # Convertir les labels en tenseur PyTorch


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/badis/MVA_deep_and_signal/Few_shots_classif/train.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
len(x_train)

30000

### Few-Shot Classification

In few-shot classification we are given a small support set of $ N $ labeled examples $ S = \{(x_1, y_1), \dots, (x_N, y_N)\} $, where each $ x_i \in \mathbb{R}^D $ is the $ D $-dimensional feature vector of an example and $ y_i \in \{1, \dots, K\} $ is the corresponding label. $ S_k $ denotes the set of examples labeled with class $ k $.

### Prototypical Networks classification

Prototypical networks compute an $ M $-dimensional representation $ c_k \in \mathbb{R}^M $, or prototype, of each class through an embedding function $ f_\phi : \mathbb{R}^D \to \mathbb{R}^M $ with learnable parameters $ \phi $. Each prototype is the mean vector of the embedded support points belonging to its class:

$$
c_k = \frac{1}{|S_k|} \sum_{(x_i, y_i) \in S_k} f_\phi(x_i) \tag{1}
$$

Given a distance function $ d : \mathbb{R}^M \times \mathbb{R}^M \to [0, +\infty) $, prototypical networks produce a distribution over classes for a query point $ x $ based on a softmax over distances to the prototypes in the embedding space:

$$
p_\phi(y = k \mid x) = \frac{\exp(-d(f_\phi(x), c_k))}{\sum_{k'} \exp(-d(f_\phi(x), c_{k'}))} \tag{2}
$$

Learning proceeds by minimizing the negative log-probability:

$$
J(\phi) = -\log p_\phi(y = k \mid x)
$$

of the true class $ k $ via stochastic gradient descent (SGD). Training episodes are formed by randomly selecting a subset of classes from the training set, then choosing a subset of examples within each class to act as the support set and a subset of the remainder to serve as query points. 


In [4]:
import train

In [ ]:
options = {
    'train_dataset_path': 'train.hdf5',  # Path to the dataset
    'experiment_root': '../output',  # Directory to save models, losses, and accuracies
    'epochs': 50,  # Number of epochs for training
    'learning_rate': 0.001,  # Learning rate for the optimizer
    'lr_scheduler_step': 20,  # Step size for the learning rate scheduler
    'lr_scheduler_gamma': 0.5,  # Multiplicative factor for learning rate decay
    'iterations': 100,  # Number of episodes per epoch
    'classes_per_it_tr': 60,  # Number of classes per training iteration
    'num_support_tr': 5,  # Number of support samples per class for training
    'num_query_tr': 5,  # Number of query samples per class for training
    'classes_per_it_val': 5,  # Number of classes per validation iteration
    'num_support_val': 5,  # Number of support samples per class for validation
    'num_query_val': 15,  # Number of query samples per class for validation
    'manual_seed': 7,  # Manual seed for reproducibility
    'device': 'cuda:0'  # Whether to use CUDA for computation
}

usage: ipykernel_launcher.py [-h] [-root DATASET_ROOT] [-exp EXPERIMENT_ROOT]
                             [-nep EPOCHS] [-lr LEARNING_RATE]
                             [-lrS LR_SCHEDULER_STEP]
                             [-lrG LR_SCHEDULER_GAMMA] [-its ITERATIONS]
                             [-cTr CLASSES_PER_IT_TR] [-nsTr NUM_SUPPORT_TR]
                             [-nqTr NUM_QUERY_TR] [-cVa CLASSES_PER_IT_VAL]
                             [-nsVa NUM_SUPPORT_VAL] [-nqVa NUM_QUERY_VAL]
                             [-seed MANUAL_SEED] [--cuda]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/badis/Library/Jupyter/runtime/kernel-v3c9bc700af36028be0c9e4062918ec4285acf5a65.json


SystemExit: 2

/Users/badis/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
train.main(options)